In [1]:
#######
#### Inspect results from cellranger run for all libraries + prepare for further processing

# Load libraries

In [2]:
import numpy as np

In [3]:
import pandas as pd 

In [4]:
import scanpy as sc

In [5]:
from numpy import savetxt 

In [6]:
import scanpy.external as sce

In [7]:
import matplotlib as matplotlib

In [8]:
import sys as sys

In [9]:
import rpy2 as rpy2
import rpy2.robjects as ro
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter

In [10]:
import os
import multiprocessing
import random
import time
import git
import sys
from datetime import date

# Load Data

## Configurations

In [11]:
data_path = '../data/current'

In [12]:
data_path

'../data/current'

In [13]:
libraries = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014']

In [14]:
libraries_text = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14']

In [15]:
libraries_dict = {'0001': 'L1', '0002': 'L2', '0003': 'L3', '0004': 'L4', '0005': 'L5',  '0006': 'L6', '0007': 'L7', '0008': 'L8', '0009': 'L9', '0010': 'L10', '0011': 'L11', '0012': 'L12', '0013': 'L13', '0014': 'L14'}

## Cellranger raw data

In [16]:
anndata_dict = dict.fromkeys(libraries)

In [17]:
for key in anndata_dict:
    dataset_path = data_path + '/preprocessed-data/pipelinerun_v21068-' + key + '/cellranger/21068-' + key +'/count_matrices/filtered_feature_bc_matrix.h5'
    print(dataset_path)
    adata_orig = sc.read_10x_h5(dataset_path, gex_only = False)
    adata_orig.var_names_make_unique()  # make variable names unique
    sc.pp.filter_genes(adata_orig, min_cells=1)  # filter genes
    anndata_dict[key]= adata_orig # save anndata in dictionary

../data/current/preprocessed-data/pipelinerun_v21068-0001/cellranger/21068-0001/count_matrices/filtered_feature_bc_matrix.h5


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


../data/current/preprocessed-data/pipelinerun_v21068-0002/cellranger/21068-0002/count_matrices/filtered_feature_bc_matrix.h5


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


../data/current/preprocessed-data/pipelinerun_v21068-0003/cellranger/21068-0003/count_matrices/filtered_feature_bc_matrix.h5


...

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


../data/current/preprocessed-data/pipelinerun_v21068-0013/cellranger/21068-0013/count_matrices/filtered_feature_bc_matrix.h5


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


../data/current/preprocessed-data/pipelinerun_v21068-0014/cellranger/21068-0014/count_matrices/filtered_feature_bc_matrix.h5


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [18]:
### Short data check

In [19]:
anndata_dict['0001']

AnnData object with n_obs × n_vars = 20572 × 24263
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'

In [20]:
anndata_dict['0001'].X.sum(axis=1)

matrix([[7099.],
        [3393.],
        [8508.],
        ...,
        [4355.],
        [4553.],
        [8805.]], dtype=float32)

In [21]:
anndata_dict = dict((libraries_dict[key], value) for (key, value) in anndata_dict.items())

In [22]:
anndata_dict['L3']

AnnData object with n_obs × n_vars = 17837 × 24329
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'

In [23]:
sys.getsizeof(anndata_dict)

640

In [24]:
#### Check out structure of all anndata objects (Nr Cells + Features)

In [25]:
for key in anndata_dict:
    print(anndata_dict[key])

AnnData object with n_obs × n_vars = 20572 × 24263
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 18667 × 24091
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 17837 × 24329
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 23187 × 24229
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 30388 × 24981
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 22541 × 24851
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 18870 × 24042
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 22824 × 24760
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with n_obs × n_vars = 18072 × 24565
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
AnnData object with

In [26]:
#### Check out features of all anndata objects for each library

In [27]:
for key in anndata_dict:
    print(key)
    print(anndata_dict[key].var['gene_ids'])

L1
AL627309.1    ENSG00000238009
AL627309.3    ENSG00000239945
AL627309.4    ENSG00000241599
AL669831.2    ENSG00000229905
AL669831.5    ENSG00000237491
                   ...       
6.1                 HTO_B0256
7.1                 HTO_B0257
8.1                 HTO_B0258
9.1                 HTO_B0259
10.1                HTO_B0260
Name: gene_ids, Length: 24263, dtype: object
L2
AL627309.1    ENSG00000238009
AL627309.3    ENSG00000239945
AL627309.4    ENSG00000241599
AL669831.2    ENSG00000229905
AL669831.5    ENSG00000237491
                   ...       
15.1                HTO_B0255
16.1                HTO_B0256
17.1                HTO_B0257
18.1                HTO_B0258
19.1                HTO_B0259
Name: gene_ids, Length: 24091, dtype: object
L3
AL627309.1    ENSG00000238009
AL627309.4    ENSG00000241599
AL669831.2    ENSG00000229905
AL669831.5    ENSG00000237491
FAM87B        ENSG00000177757
                   ...       
23.1                HTO_B0253
25.1                HTO_B0255
2

In [28]:
#### Get Value counts for each anndata object

In [29]:
for key in anndata_dict:
    print(key)
    print(anndata_dict[key].var["feature_types"].value_counts())

L1
Gene Expression     24253
Antibody Capture       10
Name: feature_types, dtype: int64
L2
Gene Expression     24083
Antibody Capture        8
Name: feature_types, dtype: int64
L3
Gene Expression     24322
Antibody Capture        7
Name: feature_types, dtype: int64
L4
Gene Expression     24219
Antibody Capture       10
Name: feature_types, dtype: int64
L5
Gene Expression     24971
Antibody Capture       10
Name: feature_types, dtype: int64
L6
Gene Expression     24841
Antibody Capture       10
Name: feature_types, dtype: int64
L7
Gene Expression     24032
Antibody Capture       10
Name: feature_types, dtype: int64
L8
Gene Expression     24750
Antibody Capture       10
Name: feature_types, dtype: int64
L9
Gene Expression     24555
Antibody Capture       10
Name: feature_types, dtype: int64
L10
Gene Expression     24615
Antibody Capture       10
Name: feature_types, dtype: int64
L11
Gene Expression     24593
Antibody Capture       10
Name: feature_types, dtype: int64
L12
Gene Expression

## Seperate RNA and protein data 

In [30]:
protein_dict = {}

In [31]:
rna_dict = {}

In [32]:
for key in anndata_dict:
    protein_dict[key] = anndata_dict[key][:, anndata_dict[key].var["feature_types"] == "Antibody Capture"].copy()

In [33]:
for key in anndata_dict:
    rna_dict[key] = anndata_dict[key][:, anndata_dict[key].var["feature_types"] == "Gene Expression"].copy()

In [34]:
protein_dict

{'L1': AnnData object with n_obs × n_vars = 20572 × 10
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L2': AnnData object with n_obs × n_vars = 18667 × 8
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L3': AnnData object with n_obs × n_vars = 17837 × 7
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L4': AnnData object with n_obs × n_vars = 23187 × 10
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L5': AnnData object with n_obs × n_vars = 30388 × 10
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L6': AnnData object with n_obs × n_vars = 22541 × 10
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L7': AnnData object with n_obs × n_vars = 18870 × 10
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L8': AnnData object with n_obs × n_vars = 22824 × 10
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L9': AnnData object with n_obs × n_vars = 18072 × 10
     var: 'gene_ids', 'feat

In [35]:
rna_dict

{'L1': AnnData object with n_obs × n_vars = 20572 × 24253
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L2': AnnData object with n_obs × n_vars = 18667 × 24083
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L3': AnnData object with n_obs × n_vars = 17837 × 24322
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L4': AnnData object with n_obs × n_vars = 23187 × 24219
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L5': AnnData object with n_obs × n_vars = 30388 × 24971
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L6': AnnData object with n_obs × n_vars = 22541 × 24841
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L7': AnnData object with n_obs × n_vars = 18870 × 24032
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L8': AnnData object with n_obs × n_vars = 22824 × 24750
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells',
 'L9': AnnData object with n_obs × n_vars = 18072 × 2455

In [36]:
##### For each library check out names of features / hashtags

In [37]:
for key in protein_dict:
    print(protein_dict[key].var_names)

Index(['1.1', '2.1', 'S3', '4.1', '5.1', '6.1', '7.1', '8.1', '9.1', '10.1'], dtype='object')
Index(['12.1', '13.1', '14.1', '15.1', '16.1', '17.1', '18.1', '19.1'], dtype='object')
Index(['21.1', '22.1', '23.1', '25.1', '26.1', '27.1', '6.1'], dtype='object')
Index(['S1', '2.2', '3.2', '4.2', 'S5', '6.2', 'S7', '8.2', '9.2', '10.2'], dtype='object')
Index(['11.2', '12.2', '13.2', '14.2', '15.2', '16.2', '7.2', '18.2', '19.2',
       '20.2'],
      dtype='object')
Index(['21.2', '22.2', '23.2', '24.2', '25.2', '26.2', '27.2', '28.2', 'S9',
       '13.2'],
      dtype='object')
Index(['11.3', '2.3', '3.3', '4.3', '5.3', '6.3', '7.3', '8.3', '9.3', 'S10'], dtype='object')
Index(['13.3', '15.3', '18.3', '19.3', '20.3', '21.3', '22.3', '23.3', '24.3',
       '25.3'],
      dtype='object')
Index(['26.3', '27.3', '28.3', '26', '28', '33', '34', '10.3', 'S9', 'S10'], dtype='object')
Index(['2.4', '3.4', 'S3', '5.4', '6.4', '7.4', '8.4', '11.4', '13.4', '15.4'], dtype='object')
Index(['Ch-CCS-

In [38]:
##### For each library add counts to protein dic as layers

In [39]:
for key in protein_dict:
    protein_dict[key].layers["counts"] = protein_dict[key].X.copy()

In [40]:
##### For each library convert protein anndatas to pandas dataframe

In [41]:
protein_dict_pd= {}

In [42]:
for key in protein_dict:
    protein_dict_pd[key] = protein_dict[key].to_df()

# Export the rna and protein data to do further analysis

##  Save protein dataset as csv for analysis in A2

In [43]:
for key in protein_dict_pd:
    data_name = data_path + '/analysis/A/' + 'A1_' + key + '_Hashtags_Barcodes_Matrix_Final.csv'
    protein_dict_pd[key].to_csv(data_name, sep = ',')

##  Save protein dataset as h5ad

In [44]:
protein_dict['L1']

AnnData object with n_obs × n_vars = 20572 × 10
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    layers: 'counts'

In [45]:
for key in protein_dict:
    data_name = data_path + '/analysis/A/' +  'A1_' + key + '_protein_Final.h5ad'
    protein_dict[key].write(data_name)

/home/icb/corinna.losert/miniconda3/envs/stark_stemi_Python/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical
/home/icb/corinna.losert/miniconda3/envs/stark_stemi_Python/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'genome' as categorical
/home/icb/corinna.losert/miniconda3/envs/stark_stemi_Python/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplac

##  Save rna dataset as h5ad

In [46]:
rna_dict['L1']

AnnData object with n_obs × n_vars = 20572 × 24253
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'

In [47]:
for key in rna_dict:
    data_name = data_path + '/analysis/A/' +  'A1_' + key + '_rna_Final.h5ad'
    rna_dict[key].write(data_name)

/home/icb/corinna.losert/miniconda3/envs/stark_stemi_Python/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical
/home/icb/corinna.losert/miniconda3/envs/stark_stemi_Python/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'genome' as categorical
/home/icb/corinna.losert/miniconda3/envs/stark_stemi_Python/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplac

## Save protein data as .RDS object

In [48]:
pandas2ri.activate()

In [49]:
saveRDS = ro.r['saveRDS']

In [50]:
for key in protein_dict_pd:
    with localconverter(ro.default_converter + pandas2ri.converter):
        r_from_pd_df = ro.conversion.py2rpy(protein_dict_pd[key])

In [51]:
for key in protein_dict_pd:
    data_name = data_path + '/analysis/A/' +  'A1_' + key + '_protein_Final.rds'
    saveRDS(protein_dict_pd[key], data_name)